In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE163721"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE163721"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE163721.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE163721.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE163721.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE163721_family.soft.gz', 'GSE163721_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE163721/GSE163721_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE163721/GSE163721_series_matrix.txt.gz
Background Information:
!Series_title	"Association of RERG Expression to Female Survival Advantage in Malignant Pleural Mesothelioma II"
!Series_summary	"Sex differences in incidence, prognosis, and treatment response have been described for many cancers. In malignant pleural mesothelioma (MPM), a lethal disease associated with asbestos exposure, men outnumber women 4 to 1, but women consistently live longer than men following surgery-based therapy. This study investigated whether tumor expression of genes associated with estrogen signaling could potentially explain observed survival differences. Two microarray datasets of MPM tumors were analyzed to discover estrogen-related genes associated with survival. A validation cohort of MPM tumors was 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset is about gene expression related to estrogen signaling in MPM
# The data is described as "microarray datasets of MPM tumors" which indicates gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# From the background information, this is a study about Malignant Pleural Mesothelioma (MPM)
# All samples appear to be tumor samples (MPM) based on the sample characteristics showing "tissue type: Tumor"
# Since all samples are mesothelioma tumors (no controls), there's no useful trait variable for association studies

# The constant feature (all samples are mesothelioma) is not useful for associative studies
trait_row = None  # No control vs. disease comparison possible
age_row = None    # No age information visible in the sample characteristics
gender_row = None # Not visible in the sample preview

# 2.2 Data Type Conversion Functions
def convert_trait(val: str) -> Optional[int]:
    """Convert trait values to binary format (0=control, 1=Mesothelioma)"""
    if val is None:
        return None
    val = val.split(':', 1)[-1].strip().lower()
    if 'mesothelioma' in val or 'mpm' in val or 'tumor' in val:
        return 1
    elif 'control' in val or 'normal' in val:
        return 0
    return None

def convert_age(val: str) -> Optional[float]:
    """Convert age values to continuous format"""
    if val is None:
        return None
    val = val.split(':', 1)[-1].strip()
    try:
        return float(val)
    except:
        return None

def convert_gender(val: str) -> Optional[int]:
    """Convert gender values to binary format (0=female, 1=male)"""
    if val is None:
        return None
    val = val.split(':', 1)[-1].strip().lower()
    if 'female' in val or 'f' == val:
        return 0
    elif 'male' in val or 'm' == val:
        return 1
    return None

# 3. Save Metadata
# Since we don't have a useful trait variable (all samples are the same), we set trait_available to False
is_trait_available = trait_row is not None

# Save initial validation information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None (as instructed)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:


No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54359
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from previous output
# The identifiers appear to be just sequential numbers (1, 2, 3...) 
# These are clearly not gene symbols and would need mapping
# They're likely probe IDs that need to be mapped to actual gene symbols

# Based on the pattern of identifiers (simple sequential numbers),
# we definitely need gene mapping

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'Block': ['1', '1', '1', '1', '1'], 'Row': [1.0, 1.0, 1.0, 1.0, 1.0], 'Column': [1.0, 2.0, 3.0, 4.0, 5.0], 'Probe Name': ['GE200017', 'GE766244', 'GE766859', 'GE519376', 'GE519777'], 'Probe Type': ['FIDUCIAL', 'DISCOVERY', 'DISCOVERY', 'DISCOVERY', 'DISCOVERY'], 'GB_ACC': [nan, 'XM_293099', 'BF588963', nan, 'BE550764'], 'Entrez Gene ID': ['-', '-', '5337', '-', '53944'], 'Gene Symbol': ['-', '-', 'PLD1', '-', 'CSNK1G1'], 'Gene Name': ['-', '-', 'phospholipase D1, phosphatidylcholine-specific', '-', 'casein kinase 1, gamma 1'], 'RefSeq ID': ['-', '-', 'NM_001130081, NM_002662', '-', 'NM_022048'], 'Unigene ID': ['-', '-', 'Hs.382865', '-', 'Hs.646508'], 'Ensembl ID': ['-', '-', 'ENSG00000075651', '-', 'ENSG00000169118'], 'UniProt ID': ['-', '-', 'Q13393, Q59EA4', '-', 'Q9HCP0'], 'Chromosome No.': ['-', '-', '3', '-', '15'], 'Chromosome Position': ['-', '-', 'chr3:171318619-171528273', '-', 'chr15:64457717-64648442'], 'GO biologic

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the appropriate columns for mapping
# The gene expression data shows row indices as numeric IDs like 1, 2, 3...
# In the gene annotation dataframe, 'ID' column contains these same identifiers
# The 'Gene Symbol' column in the annotation dataframe contains the gene symbols we need

# Checking column names to confirm
print("Column names in gene annotation dataframe:")
print(gene_annotation.columns.tolist())

# 2. Get gene mapping dataframe by extracting the two columns from the gene annotation dataframe
# Extract ID and Gene Symbol columns for mapping
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# Display summary of the mapping data
print("\nGene mapping summary:")
print(f"Total probes in mapping: {len(gene_mapping)}")
print(f"Probes with gene symbols: {len(gene_mapping[gene_mapping['Gene'] != '-'])}")
print(f"Sample of gene mapping data:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
# Apply the mapping to convert probes to genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Display summary of the resulting gene expression data
print("\nGene expression data summary:")
print(f"Shape: {gene_data.shape}")
print("First few rows and columns:")
print(gene_data.iloc[:5, :5])


Column names in gene annotation dataframe:
['ID', 'Block', 'Row', 'Column', 'Probe Name', 'Probe Type', 'GB_ACC', 'Entrez Gene ID', 'Gene Symbol', 'Gene Name', 'RefSeq ID', 'Unigene ID', 'Ensembl ID', 'UniProt ID', 'Chromosome No.', 'Chromosome Position', 'GO biological process', 'GO molecular function', 'GO cellular component', 'SEQUENCE', 'SPOT_ID']

Gene mapping summary:
Total probes in mapping: 54359
Probes with gene symbols: 33916
Sample of gene mapping data:
  ID     Gene
0  1        -
1  2        -
2  3     PLD1
3  4        -
4  5  CSNK1G1



Gene expression data summary:
Shape: (16896, 46)
First few rows and columns:
        GSM4984502  GSM4984503  GSM4984504  GSM4984505  GSM4984506
Gene                                                              
A1BG      0.453598    0.867600    1.025219    0.847260    0.890790
A1CF      0.813705    1.519490    0.954572    0.542159    1.383813
A2LD1     1.387532    4.196274    1.546933    2.156707    4.564961
A2ML1     1.600246    1.708296    1.472621    1.092117    1.479736
A4GALT    0.700037    0.348783    0.462628    0.229904    0.352832


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample IDs in the gene expression data
# Since all samples are from tumor tissue, they all have mesothelioma (but this is not useful as a trait)
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Extract gender information from the original matrix file
gender_data = {}
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Read the matrix file to extract sample characteristics
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Display unique values in each row of clinical data
    characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample characteristics dictionary:")
    print(characteristics_dict)
    
    # Try to find gender information in the characteristics
    gender_row = None
    for idx, values in characteristics_dict.items():
        if any('sex:' in str(v).lower() for v in values):
            gender_row = idx
            break
    
    if gender_row is not None:
        # Extract gender data from the row
        for col in clinical_data.columns:
            if col != '!Sample_geo_accession':
                continue
                
            for idx, row in clinical_data.iterrows():
                if idx == gender_row:
                    for i, sample_id in enumerate(clinical_data.iloc[0].values):
                        if i > 0 and sample_id in sample_ids:  # Skip the first column (header)
                            gender_val = clinical_data.iloc[idx, i]
                            if isinstance(gender_val, str) and 'sex:' in gender_val.lower():
                                gender = 0 if 'f' in gender_val.lower() else 1 if 'm' in gender_val.lower() else None
                                gender_data[sample_id] = gender
except Exception as e:
    print(f"Error extracting gender data: {e}")

# Create a clinical dataframe
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df['Mesothelioma'] = 1  # All samples have mesothelioma

# Add gender if available
if gender_data:
    clinical_df['Gender'] = clinical_df.index.map(lambda x: gender_data.get(x))
    print(f"Added gender data for {sum(pd.notna(clinical_df['Gender']))} samples")

print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data (transpose gene expression data to have samples as rows)
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Mesothelioma')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait is biased (it will be since all samples are mesothelioma)
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Mesothelioma')

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # We do have trait data, it's just that all values are the same
    is_biased=is_trait_biased,  # This will be True since all samples have the same trait value
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from mesothelioma patients only (no controls), making trait biased."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to biased trait variable. Final linked data not saved.")

Shape of gene data after normalization: (16665, 46)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE163721.csv
Number of samples: 46
Sample characteristics dictionary:
{0: ['tissue type: Tumor']}
Clinical data preview:
            Mesothelioma
GSM4984502             1
GSM4984503             1
GSM4984504             1
GSM4984505             1
GSM4984506             1
Saved clinical data to ../../output/preprocess/Mesothelioma/clinical_data/GSE163721.csv
Shape of linked data: (46, 16666)


Shape of linked data after handling missing values: (46, 16666)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Dataset validation failed due to biased trait variable. Final linked data not saved.
